<a href="https://colab.research.google.com/github/Jyun1998/LG-Meeting-Transcript-Summarization/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install rouge
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

     |████████████████████████████████| 2.9 MB 6.4 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 66.5 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-2b6vn6ty/kobart_9a6f597d10534bd1bfd3487d124df2d1
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-install-2b6vn6ty/kobart_9a6f597d10534bd1bfd3487d124df2d1
     |████████████████████████████████| 1.9 MB 8.4 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
  Created wheel for kobart: filename=kobart-0.4-py3-none-any.whl size=8538 sha256=d3e6dd341a1d63f3e67672576b73d67a31e73e4e189969161d8cbef898cf6022
  Stored in directory: /tmp/pip-ephem-wheel-cache-cs1b5l_u/w

In [ ]:
from transformers import BartModel
from transformers import BartForConditionalGeneration, BertTokenizer
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np

import os
import json

import gc

from tqdm import tqdm

In [ ]:
DIR = "/content/drive/MyDrive/dacon/lg/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
tokenizer = get_kobart_tokenizer()

max_seq_len = 900
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 4

[██████████████████████████████████████████████████]


In [ ]:
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/dacon/lg/team/epoch=02-val_loss=0.748')
model = model.to(device)
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   

In [ ]:
def summarize(text, max_seq_len):
    
    context_tokens = ['<s>'] + tokenizer.tokenize(text) + ['</s>']
    input_ids = tokenizer.convert_tokens_to_ids(context_tokens) 

    if len(input_ids) < max_seq_len:   
            while len(input_ids) < max_seq_len: 
                input_ids += [tokenizer.pad_token_id] 
    else:
        input_ids = input_ids[:max_seq_len - 1] + [tokenizer.eos_token_id]

    res_ids = model.generate(torch.tensor([input_ids]).to(device),
                                        max_length=max_seq_len,
                                        num_beams=5,
                                        temperature = 0.5,
                                        top_k = -1,
                                        top_p = -1,
                                        no_repeat_ngram_size = 3,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bad_words_ids=[[tokenizer.unk_token_id]])        
    res = tokenizer.batch_decode(res_ids.tolist())[0]
    return res.replace('<s>', '').replace('</s>', '').replace('<usr> ', '')

ans = []
for i in tqdm(range(test.shape[0])):
  ans.append(summarize(test['total'].iloc[i], max_seq_len))

100%|██████████| 506/506 [04:17<00:00,  1.97it/s]


In [ ]:
submissions = pd.read_csv(os.path.join(DIR, "sample_submission.csv"))
submissions['summary'] = ans
submissions.to_csv(os.path.join(DIR, "final_sub.csv"), index=False, encoding="utf-8-sig")

In [ ]:
from google.colab import files
files.download(os.path.join(DIR, "final_sub.csv"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>